In [1]:
import open3d as o3d
import numpy as np
import copy
from scipy import interpolate


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [51]:
# Load the intraoral mesh
intraoral_mesh = o3d.io.read_triangle_mesh(r'D:\sunny\Codes\DPS\data_teethseg\origin\001502_origin.ply')
intraoral_mesh.compute_vertex_normals()
n_sample_pts = 32

axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50, origin=[0,0,0])


### Recenter the mesh (later the labelled one as well)

In [52]:
# Find two end points of the intraoral mesh by the max and min theta values
def find_key_points(mesh):
    vertices = np.asarray(mesh.vertices)
    x,y,z = vertices[:,0], vertices[:,1], vertices[:,2]
    theta = np.arctan2(z,x) + np.pi/2
    theta = np.where(theta<0, theta+2*np.pi, theta) # if theta < 0, add 2pi to make it positive
    endpt1 = vertices[np.argmin(theta)]
    endpt2 = vertices[np.argmax(theta)]
    centre = (endpt1 + endpt2) / 2
    return endpt1, endpt2, centre

# Recentre the mesh with centre keypoint at the origin, rotate the mesh about y axis such that x-axis point to endpt1
def recentre_mesh(mesh, key_points):
    endpt1, endpt2, centre_keypt = key_points
    vertices = np.asarray(mesh.vertices)
    vertices = vertices - centre_keypt
    endpt1 = endpt1 - centre_keypt
    endpt2 = endpt2 - centre_keypt


    # Rotate the mesh about y axis such that x-axis point to endpt1
    theta = np.arctan2(endpt1[2], endpt1[0])
    # print(f"endpt1: {endpt1}, endpt2: {endpt2}, theta: {theta}")
    R = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    vertices = np.dot(vertices, R.T)
    endpt1 = np.dot(endpt1, R.T)
    endpt2 = np.dot(endpt2, R.T)
    print(f"endpt1: {endpt1}, endpt2: {endpt2}")
    assert endpt1[2] < 1e-6, f"end point 1 {endpt1} is not on x-axis"
    assert endpt2[2] < 1e-6, f"end point 2 {endpt2} is not on x-axis"  

    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    return mesh, endpt1, endpt2

In [53]:
# Test the recentre_mesh function
keypts = find_key_points(intraoral_mesh)

# Visualize the original mesh with key points
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(keypts)
pc.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([intraoral_mesh, axes, pc])

In [54]:
intraoral_mesh_rc, endpt1_rc, endpt2_rc = recentre_mesh(intraoral_mesh, keypts) # rc: re-centered

# Visualize the recentred mesh with key points, with axis drawn
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector([endpt1_rc, endpt2_rc])
pc.paint_uniform_color([1, 0, 0])

o3d.visualization.draw_geometries([intraoral_mesh_rc, axes, pc])

endpt1: [22.42717787  0.33631639  0.        ], endpt2: [-22.42717787  -0.33631639   0.        ]


In [58]:
""" Express all mesh vertices in polar coordinates (r, theta) in x-z plane
    r: the distance to the origin
    theta: the angle to x-axis"""

vertices = np.asarray(intraoral_mesh_rc.vertices)
r = np.linalg.norm(vertices[:,[0,2]], axis=1)
theta = np.arctan2(vertices[:,2], vertices[:,0])



# Create the outer and inner half-ellipse curves
# Compute the major and minor axis of the ellipse

# a = endpt1_rc[0] # major axis of the ellipse
end_mask = np.logical_or(theta < 0.01*np.pi, theta > 0.99*np.pi) # Extract out points near the x axis (within theta value of [0.01pi, 0.99pi])
print(f"Number of points near x-axis: {np.sum(end_mask)}")
r_end = r[end_mask]
a = np.mean(r_end) # major axis of the ellipse

middle_mask = np.logical_and(theta > 0.49*np.pi, theta < 0.51*np.pi) # Extract out points near the z axis (within theta value of [0.48pi, 0.52pi])
print(f"Number of points near z-axis: {np.sum(middle_mask)}")
r_middle = r[middle_mask]
b = np.mean(r_middle) # minor axis of the ellipse

a_outer = a * 1.4
b_outer = b * 1.1
a_inner = a * 0.7
b_inner = b * 0.8



Number of points near x-axis: 245
Number of points near z-axis: 328


In [59]:
def ellipse_coordinates(a, b, theta):
    """ return an array of x, z coordinates of points lying on the ellipse with major axis a, minor axis b, and angles theta"""
    r = a * b / np.sqrt((b * np.cos(theta))**2 + (a * np.sin(theta))**2)
    x = r * np.cos(theta)
    z = r * np.sin(theta)
    y = np.zeros_like(x)
    # rearrange into (x, y, z) format for all points (i.e. shape (n, 3))
    coordinates = np.vstack((x, y, z)).T
    return coordinates

In [61]:
# visualize the outer and inner half-ellipse curves with original mesh
theta_values = np.linspace(0, np.pi, 256)
outer_ellipse_pts = ellipse_coordinates(a_outer, b_outer, theta_values)
inner_ellipse_pts = ellipse_coordinates(a_inner, b_inner, theta_values)
print(f"outer_ellipse_pts: {outer_ellipse_pts.shape}, inner_ellipse_pts: {inner_ellipse_pts.shape}")

outer_ellipse_mesh = o3d.geometry.PointCloud()
outer_ellipse_mesh.points = o3d.utility.Vector3dVector(outer_ellipse_pts)
outer_ellipse_mesh.paint_uniform_color([0, 1, 0])


# FIXME: maybe the inner ellipse is not necessary!!???
inner_ellipse_mesh = o3d.geometry.PointCloud()
inner_ellipse_mesh.points = o3d.utility.Vector3dVector(inner_ellipse_pts)
inner_ellipse_mesh.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([intraoral_mesh_rc, outer_ellipse_mesh, inner_ellipse_mesh, axes])


outer_ellipse_pts: (256, 3), inner_ellipse_pts: (256, 3)
